# RESEARCH QUESTION 2 - Cyclone Path

### SETUP

In [1]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
import geopandas as gpd
import movingpandas as mpd
from shapely.geometry import Point
import plotly.express as px
import matplotlib.pyplot as plt
plt.style.use("ggplot")
import torch
from darts import TimeSeries
from darts.utils.callbacks import TFMProgressBar
from darts.models import (NBEATSModel)
from darts.metrics import mae, rmse, mse
from darts.dataprocessing.transformers import Scaler
import logging
logging.disable(logging.CRITICAL)
from sklearn.metrics import mean_squared_error, mean_absolute_error
from tabulate import tabulate

### DATA PREPROCESSING

In [2]:
### DATA PREPROCESSING

cyc = pd.read_csv('IDCKMSTM0S.csv', skiprows=3)

cyc.columns.to_list()

cyc1 = cyc[['NAME',
            'DISTURBANCE_ID',
            'TM',
            'LAT',
            'LON',
            'CENTRAL_PRES',
            'ENV_PRES',
            'MN_RADIUS_GF_SECNE',
            'MN_RADIUS_GF_SECSE',
            'MN_RADIUS_GF_SECSW',
            'MN_RADIUS_GF_SECNW']]

sub1 = cyc1.loc[(cyc1['DISTURBANCE_ID']== 'AU199899_09U')]

# Remove white space value from all object columns
obcols = cyc1.select_dtypes(object).columns
cyc1[obcols] = cyc1[obcols].apply(lambda x: x.replace(" ",""))

# Remove negative signs in LON column
cyc1['LON'] = cyc1['LON'].apply(lambda x: x.replace("-",""))

# Remove trailing white spaces from string values
cyc1[['NAME',
      'DISTURBANCE_ID',
      'TM']] = cyc1[['NAME',
                     'DISTURBANCE_ID',
                     'TM']].apply(lambda x: x.str.strip())

# Convert dtypes from object to numeric
cyc1[['LAT',
      'LON',
      'CENTRAL_PRES',
      'ENV_PRES',
      'MN_RADIUS_GF_SECNE',
      'MN_RADIUS_GF_SECSE',
      'MN_RADIUS_GF_SECSW',
      'MN_RADIUS_GF_SECNW']] = cyc1[['LAT',
                                     'LON',
                                     'CENTRAL_PRES',
                                     'ENV_PRES',
                                     'MN_RADIUS_GF_SECNE',
                                     'MN_RADIUS_GF_SECSE',
                                     'MN_RADIUS_GF_SECSW',
                                     'MN_RADIUS_GF_SECNW']].apply(lambda x:pd.to_numeric(x))

# Convert TM to datetime
cyc1['TM'] = pd.to_datetime(cyc1['TM'], dayfirst=True)

cyc2 = cyc1.copy()

# Remove all observations that have missing values in TM
cyc2 = cyc2[cyc2['TM'].notnull()]

# Resample to 6 hourly interval
cyc2 = cyc2.set_index('TM').groupby(['DISTURBANCE_ID', 'NAME']).resample('6H').mean()

cyc3= cyc2.reset_index(level=['TM', 'NAME'])

# interpolate 'inside' valid values for each group
groupunique = cyc3.index.unique().to_list()
for i in groupunique:
    cyc3.loc[[i],['LAT',
                  'LON',
                  'CENTRAL_PRES',
                  'ENV_PRES',
                  'MN_RADIUS_GF_SECNE',
                  'MN_RADIUS_GF_SECSE',
                  'MN_RADIUS_GF_SECSW',
                  'MN_RADIUS_GF_SECNW']] = cyc3.loc[[i],['LAT',
                                                         'LON',
                                                         'CENTRAL_PRES',
                                                         'ENV_PRES',
                                                         'MN_RADIUS_GF_SECNE',
                                                         'MN_RADIUS_GF_SECSE',
                                                         'MN_RADIUS_GF_SECSW',
                                                         'MN_RADIUS_GF_SECNW']].interpolate(limit=20,
                                                                                            limit_area='inside')

cyc3 = cyc3.reset_index()

# drop all rows with at least 2 NAs in R34s
cyc3 = cyc3.drop(cyc3.loc[sum([(cyc3['MN_RADIUS_GF_SECNE'].isnull()),
                               (cyc3['MN_RADIUS_GF_SECSE'].isnull()),
                               (cyc3['MN_RADIUS_GF_SECSW'].isnull()),
                               (cyc3['MN_RADIUS_GF_SECNW'].isnull())])>=2].index)

# drop all rows with at least 2 NAs
cyc3 = cyc3.drop(cyc3.loc[sum([(cyc3['MN_RADIUS_GF_SECNE'].isnull()),
                               (cyc3['MN_RADIUS_GF_SECSE'].isnull()),
                               (cyc3['MN_RADIUS_GF_SECSW'].isnull()),
                               (cyc3['MN_RADIUS_GF_SECNW'].isnull()),
                               (cyc3['ENV_PRES'].isnull())])>=2].index)

# kNN-neighbour imputation
from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=3)
cyc3_imp = imputer.fit_transform(cyc3[['ENV_PRES',
                                       'MN_RADIUS_GF_SECNE',
                                       'MN_RADIUS_GF_SECSE',
                                       'MN_RADIUS_GF_SECSW',
                                       'MN_RADIUS_GF_SECNW']])

cyc3_imp = pd.DataFrame(cyc3_imp, index= cyc3.index,
                        columns=['ENV_PRES',
                                 'MN_RADIUS_GF_SECNE',
                                 'MN_RADIUS_GF_SECSE',
                                 'MN_RADIUS_GF_SECSW',
                                 'MN_RADIUS_GF_SECNW'])

cyc4 = cyc3[['LAT',
             'LON',
             'CENTRAL_PRES']]

cyc4[['ENV_PRES',
      'MN_RADIUS_GF_SECNE',
      'MN_RADIUS_GF_SECSE',
      'MN_RADIUS_GF_SECSW',
      'MN_RADIUS_GF_SECNW']] = cyc3_imp

cyc4 = cyc3.set_index('TM').groupby(['DISTURBANCE_ID']).resample('6H').interpolate()
cyc4 = cyc4.reset_index(level='TM')

cyc4[['NAME', 'DISTURBANCE_ID']] = cyc4[['NAME', 'DISTURBANCE_ID']].ffill()

cyc4 = cyc4.dropna()
cyc4 = cyc4.reset_index(drop=True)

# only keep cyclone groups with at least 3 observations
s = cyc4.groupby('DISTURBANCE_ID').size() >= 4
cyc5 = cyc4.loc[cyc4['DISTURBANCE_ID'].isin(s[s].index)]

cyc5.to_csv('cyc5.csv', index=False)

################################################################################
# Split data based on whether a cyclone ever exists East or West of a
#  particular line of Longitude

# Only keep cyclones that travel West of a particular Longitude (threshold_LON)
threshold_LON = 131 # Approx Longitude of Darwin
# Obtain UID of cyclones to keep in West
UIDW = cyc5.groupby('DISTURBANCE_ID').filter(lambda x: (x['LON'] < threshold_LON).any())
UIDW = UIDW['DISTURBANCE_ID'].unique()
# Filter dataframe
cyc5W = cyc5[cyc5['DISTURBANCE_ID'].isin(UIDW)]
cyc5W.to_csv('cyc5W.csv', index=False)

# Only keep cyclones that travel East of a particular Longitude (threshold_LON)
# Obtain UID of cyclones to keep in East
UIDE = cyc5.groupby('DISTURBANCE_ID').filter(lambda x: (x['LON'] > threshold_LON).any())
UIDE = UIDE['DISTURBANCE_ID'].unique()
# Filter dataframe
cyc5E = cyc5[cyc5['DISTURBANCE_ID'].isin(UIDE)]
cyc5E.to_csv('cyc5E.csv', index=False)
################################################################################

################################################################################
# Generate lists of dataframes: Total, East region only and West region only
cyc6 = cyc5.groupby('DISTURBANCE_ID')           # For RQ1 and RQ2
#cyc6 = cyc5W.groupby('DISTURBANCE_ID')         # For RQ3
#cyc6 = cyc5E.groupby('DISTURBANCE_ID')         # For RQ3
################################################################################

# Write each group as a record of dictionary named d
d = dict()
for k in list(cyc6.groups.keys()):
    dfname = str(k)
    d[dfname] = cyc6.get_group(k)

# get data frames from dictionary d
df_list = []
for i in list(cyc6.groups.keys()):
    df = 'd["' + str(i) + '"]'
    df_list.append(df)

################################################################################
# Assign dataframes to corresponding names
### For RQ1 and RQ2 ###
# All
(AU199697_14U,AU199899_05U,AU199900_13U,AU200001_06U,AU200203_01U,AU200203_02U,
 AU200203_03U,AU200203_04U,AU200203_06U,AU200203_07U,AU200304_01U,AU200304_02U,
 AU200304_03U,AU200304_04U,AU200304_05U,AU200304_07U,AU200304_08U,AU200304_09U,
 AU200304_11U,AU200405_01U,AU200405_02U,AU200405_03U,AU200405_04U,AU200405_05U,
 AU200405_07U,AU200405_08U,AU200405_09U,AU200405_10U,AU200506_01U,AU200506_05U,
 AU200506_06U,AU200506_08U,AU200506_10U,AU200506_14U,AU200506_15U,AU200506_16U,
 AU200506_17U,AU200506_20U,AU200506_22U,AU200607_05U,AU200607_11U,AU200607_12U,
 AU200607_13U,AU200607_16U,AU200708_01U,AU200708_03U,AU200708_06U,AU200708_08U,
 AU200708_11U,AU200708_15U,AU200708_20U,AU200708_22U,AU200708_23U,AU200809_02U,
 AU200809_03U,AU200809_06U,AU200809_08U,AU200809_10U,AU200809_17U,AU200809_18U,
 AU200809_23U,AU200910_01U,AU200910_06U,AU200910_07U,AU200910_09U,AU200910_11U,
 AU200910_12U,AU200910_13U,AU201011_02U,AU201011_09U,AU201011_10U,AU201011_11U,
 AU201011_12U,AU201011_14U,AU201011_16U,AU201011_17U,AU201011_29U,AU201112_01U,
 AU201112_04U,AU201112_07U,AU201112_11U,AU201112_12U,AU201112_15U,AU201112_16U,
 AU201213_03U,AU201213_04U,AU201213_05U,AU201213_10U,AU201213_13U,AU201213_14U,
 AU201213_17U,AU201213_18U,AU201314_01U,AU201314_03U,AU201314_04U,AU201314_07U,
 AU201314_09U,AU201314_10U,AU201314_14U,AU201314_15U,AU201314_16U,AU201415_04U,
 AU201415_13U,AU201415_14U,AU201415_16U,AU201415_17U,AU201415_19U,AU201415_21U,
 AU201415_24U,AU201516_08U,AU201516_09U,AU201516_10U,AU201617_07U,AU201617_19U,
 AU201617_20U,AU201617_23U,AU201617_24U,AU201617_26U,AU201617_29U,AU201617_30U,
 AU201718_02U,AU201718_03U,AU201718_06U,AU201718_17U,AU201718_20U,AU201718_22U,
 AU201718_24U,AU201819_04U,AU201819_05U,AU201819_07U,AU201819_12U,AU201819_14U,
 AU201819_17U,AU201819_19U,AU201819_20U,AU201819_21U,AU201819_25U,AU201819_26U,
 AU201920_03U,AU201920_05U,AU201920_06U,AU201920_07U,AU201920_08U,AU201920_10U,
 AU201920_12U,AU202021_07U,AU202021_09U,AU202021_10U,AU202021_11U,AU202021_15U,
 AU202021_17U,AU202021_22U,AU202021_23U,AU202122_02U,AU202122_05U,AU202122_07U,
 AU202122_08U,AU202122_10U,AU202122_18U,AU202122_22U,AU202122_23U,AU202122_27U,
 AU202122_28U,AU202122_36U,AU202223_01U,AU202223_05U,AU202223_13U,AU202223_14U,
 AU202223_21U,AU202223_23U,AU202324_02U,AU202324_04U,AU202324_05U,AU202324_08U,
 AU202324_09U,AU202324_11U,
 AU202324_13U) = (d["AU199697_14U"],d["AU199899_05U"],d["AU199900_13U"],d["AU200001_06U"],
                  d["AU200203_01U"],d["AU200203_02U"],d["AU200203_03U"],d["AU200203_04U"],
                  d["AU200203_06U"],d["AU200203_07U"],d["AU200304_01U"],d["AU200304_02U"],
                  d["AU200304_03U"],d["AU200304_04U"],d["AU200304_05U"],d["AU200304_07U"],
                  d["AU200304_08U"],d["AU200304_09U"],d["AU200304_11U"],d["AU200405_01U"],
                  d["AU200405_02U"],d["AU200405_03U"],d["AU200405_04U"],d["AU200405_05U"],
                  d["AU200405_07U"],d["AU200405_08U"],d["AU200405_09U"],d["AU200405_10U"],
                  d["AU200506_01U"],d["AU200506_05U"],d["AU200506_06U"],d["AU200506_08U"],
                  d["AU200506_10U"],d["AU200506_14U"],d["AU200506_15U"],d["AU200506_16U"],
                  d["AU200506_17U"],d["AU200506_20U"],d["AU200506_22U"],d["AU200607_05U"],
                  d["AU200607_11U"],d["AU200607_12U"],d["AU200607_13U"],d["AU200607_16U"],
                  d["AU200708_01U"],d["AU200708_03U"],d["AU200708_06U"],d["AU200708_08U"],
                  d["AU200708_11U"],d["AU200708_15U"],d["AU200708_20U"],d["AU200708_22U"],
                  d["AU200708_23U"],d["AU200809_02U"],d["AU200809_03U"],d["AU200809_06U"],
                  d["AU200809_08U"],d["AU200809_10U"],d["AU200809_17U"],d["AU200809_18U"],
                  d["AU200809_23U"],d["AU200910_01U"],d["AU200910_06U"],d["AU200910_07U"],
                  d["AU200910_09U"],d["AU200910_11U"],d["AU200910_12U"],d["AU200910_13U"],
                  d["AU201011_02U"],d["AU201011_09U"],d["AU201011_10U"],d["AU201011_11U"],
                  d["AU201011_12U"],d["AU201011_14U"],d["AU201011_16U"],d["AU201011_17U"],
                  d["AU201011_29U"],d["AU201112_01U"],d["AU201112_04U"],d["AU201112_07U"],
                  d["AU201112_11U"],d["AU201112_12U"],d["AU201112_15U"],d["AU201112_16U"],
                  d["AU201213_03U"],d["AU201213_04U"],d["AU201213_05U"],d["AU201213_10U"],
                  d["AU201213_13U"],d["AU201213_14U"],d["AU201213_17U"],d["AU201213_18U"],
                  d["AU201314_01U"],d["AU201314_03U"],d["AU201314_04U"],d["AU201314_07U"],
                  d["AU201314_09U"],d["AU201314_10U"],d["AU201314_14U"],d["AU201314_15U"],
                  d["AU201314_16U"],d["AU201415_04U"],d["AU201415_13U"],d["AU201415_14U"],
                  d["AU201415_16U"],d["AU201415_17U"],d["AU201415_19U"],d["AU201415_21U"],
                  d["AU201415_24U"],d["AU201516_08U"],d["AU201516_09U"],d["AU201516_10U"],
                  d["AU201617_07U"],d["AU201617_19U"],d["AU201617_20U"],d["AU201617_23U"],
                  d["AU201617_24U"],d["AU201617_26U"],d["AU201617_29U"],d["AU201617_30U"],
                  d["AU201718_02U"],d["AU201718_03U"],d["AU201718_06U"],d["AU201718_17U"],
                  d["AU201718_20U"],d["AU201718_22U"],d["AU201718_24U"],d["AU201819_04U"],
                  d["AU201819_05U"],d["AU201819_07U"],d["AU201819_12U"],d["AU201819_14U"],
                  d["AU201819_17U"],d["AU201819_19U"],d["AU201819_20U"],d["AU201819_21U"],
                  d["AU201819_25U"],d["AU201819_26U"],d["AU201920_03U"],d["AU201920_05U"],
                  d["AU201920_06U"],d["AU201920_07U"],d["AU201920_08U"],d["AU201920_10U"],
                  d["AU201920_12U"],d["AU202021_07U"],d["AU202021_09U"],d["AU202021_10U"],
                  d["AU202021_11U"],d["AU202021_15U"],d["AU202021_17U"],d["AU202021_22U"],
                  d["AU202021_23U"],d["AU202122_02U"],d["AU202122_05U"],d["AU202122_07U"],
                  d["AU202122_08U"],d["AU202122_10U"],d["AU202122_18U"],d["AU202122_22U"],
                  d["AU202122_23U"],d["AU202122_27U"],d["AU202122_28U"],d["AU202122_36U"],
                  d["AU202223_01U"],d["AU202223_05U"],d["AU202223_13U"],d["AU202223_14U"],
                  d["AU202223_21U"],d["AU202223_23U"],d["AU202324_02U"],d["AU202324_04U"],
                  d["AU202324_05U"],d["AU202324_08U"],d["AU202324_09U"],d["AU202324_11U"],
                  d["AU202324_13U"])
df_all = [AU199697_14U,AU199899_05U,AU199900_13U,AU200001_06U,AU200203_01U,AU200203_02U,
          AU200203_03U,AU200203_04U,AU200203_06U,AU200203_07U,AU200304_01U,AU200304_02U,
          AU200304_03U,AU200304_04U,AU200304_05U,AU200304_07U,AU200304_08U,AU200304_09U,
          AU200304_11U,AU200405_01U,AU200405_02U,AU200405_03U,AU200405_04U,AU200405_05U,
          AU200405_07U,AU200405_08U,AU200405_09U,AU200405_10U,AU200506_01U,AU200506_05U,
          AU200506_06U,AU200506_08U,AU200506_10U,AU200506_14U,AU200506_15U,AU200506_16U,
          AU200506_17U,AU200506_20U,AU200506_22U,AU200607_05U,AU200607_11U,AU200607_12U,
          AU200607_13U,AU200607_16U,AU200708_01U,AU200708_03U,AU200708_06U,AU200708_08U,
          AU200708_11U,AU200708_15U,AU200708_20U,AU200708_22U,AU200708_23U,AU200809_02U,
          AU200809_03U,AU200809_06U,AU200809_08U,AU200809_10U,AU200809_17U,AU200809_18U,
          AU200809_23U,AU200910_01U,AU200910_06U,AU200910_07U,AU200910_09U,AU200910_11U,
          AU200910_12U,AU200910_13U,AU201011_02U,AU201011_09U,AU201011_10U,AU201011_11U,
          AU201011_12U,AU201011_14U,AU201011_16U,AU201011_17U,AU201011_29U,AU201112_01U,
          AU201112_04U,AU201112_07U,AU201112_11U,AU201112_12U,AU201112_15U,AU201112_16U,
          AU201213_03U,AU201213_04U,AU201213_05U,AU201213_10U,AU201213_13U,AU201213_14U,
          AU201213_17U,AU201213_18U,AU201314_01U,AU201314_03U,AU201314_04U,AU201314_07U,
          AU201314_09U,AU201314_10U,AU201314_14U,AU201314_15U,AU201314_16U,AU201415_04U,
          AU201415_13U,AU201415_14U,AU201415_16U,AU201415_17U,AU201415_19U,AU201415_21U,
          AU201415_24U,AU201516_08U,AU201516_09U,AU201516_10U,AU201617_07U,AU201617_19U,
          AU201617_20U,AU201617_23U,AU201617_24U,AU201617_26U,AU201617_29U,AU201617_30U,
          AU201718_02U,AU201718_03U,AU201718_06U,AU201718_17U,AU201718_20U,AU201718_22U,
          AU201718_24U,AU201819_04U,AU201819_05U,AU201819_07U,AU201819_12U,AU201819_14U,
          AU201819_17U,AU201819_19U,AU201819_20U,AU201819_21U,AU201819_25U,AU201819_26U,
          AU201920_03U,AU201920_05U,AU201920_06U,AU201920_07U,AU201920_08U,AU201920_10U,
          AU201920_12U,AU202021_07U,AU202021_09U,AU202021_10U,AU202021_11U,AU202021_15U,
          AU202021_17U,AU202021_22U,AU202021_23U,AU202122_02U,AU202122_05U,AU202122_07U,
          AU202122_08U,AU202122_10U,AU202122_18U,AU202122_22U,AU202122_23U,AU202122_27U,
          AU202122_28U,AU202122_36U,AU202223_01U,AU202223_05U,AU202223_13U,AU202223_14U,
          AU202223_21U,AU202223_23U,AU202324_02U,AU202324_04U,AU202324_05U,AU202324_08U,
          AU202324_09U,AU202324_11U,AU202324_13U]
### For RQ3 ###
# West region only
(AU199697_14U,AU199899_05U,AU199900_13U,AU200203_02U,AU200203_03U,AU200203_04U,AU200203_06U,
 AU200203_07U,AU200304_01U,AU200304_03U,AU200304_05U,AU200304_07U,AU200304_08U,AU200304_11U,
 AU200405_01U,AU200405_02U,AU200405_03U,AU200405_04U,AU200405_07U,AU200405_08U,AU200405_09U,
 AU200506_01U,AU200506_05U,AU200506_06U,AU200506_14U,AU200506_16U,AU200506_20U,AU200607_11U,
 AU200607_12U,AU200607_13U,AU200708_01U,AU200708_06U,AU200708_08U,AU200708_11U,AU200708_15U,
 AU200708_20U,AU200708_22U,AU200708_23U,AU200809_02U,AU200809_03U,AU200809_08U,AU200809_10U,
 AU200809_18U,AU200910_01U,AU200910_06U,AU200910_12U,AU200910_13U,AU201011_02U,AU201011_09U,
 AU201011_12U,AU201011_16U,AU201011_17U,AU201011_29U,AU201112_01U,AU201112_07U,AU201112_11U,
 AU201112_15U,AU201112_16U,AU201213_04U,AU201213_05U,AU201213_10U,AU201213_17U,AU201314_01U,
 AU201314_03U,AU201314_04U,AU201314_09U,AU201314_14U,AU201314_16U,AU201415_04U,AU201415_16U,
 AU201415_19U,AU201415_21U,AU201516_08U,AU201516_09U,AU201617_07U,AU201617_20U,AU201617_23U,
 AU201617_26U,AU201617_29U,AU201617_30U,AU201718_02U,AU201718_03U,AU201718_06U,AU201718_17U,
 AU201718_20U,AU201819_05U,AU201819_12U,AU201819_17U,AU201819_19U,AU201819_21U,AU201819_25U,
 AU201920_03U,AU201920_05U,AU201920_08U,AU202021_07U,AU202021_10U,AU202021_15U,AU202021_22U,
 AU202021_23U,AU202122_02U,AU202122_05U,AU202122_22U,AU202122_23U,AU202122_27U,AU202122_28U,
 AU202122_36U,AU202223_01U,AU202223_05U,AU202223_13U,AU202223_21U,AU202223_23U,AU202324_04U,
 AU202324_08U,
 AU202324_11U) = (d["AU199697_14U"],d["AU199899_05U"],d["AU199900_13U"],d["AU200203_02U"],
                  d["AU200203_03U"],d["AU200203_04U"],d["AU200203_06U"],d["AU200203_07U"],
                  d["AU200304_01U"],d["AU200304_03U"],d["AU200304_05U"],d["AU200304_07U"],
                  d["AU200304_08U"],d["AU200304_11U"],d["AU200405_01U"],d["AU200405_02U"],
                  d["AU200405_03U"],d["AU200405_04U"],d["AU200405_07U"],d["AU200405_08U"],
                  d["AU200405_09U"],d["AU200506_01U"],d["AU200506_05U"],d["AU200506_06U"],
                  d["AU200506_14U"],d["AU200506_16U"],d["AU200506_20U"],d["AU200607_11U"],
                  d["AU200607_12U"],d["AU200607_13U"],d["AU200708_01U"],d["AU200708_06U"],
                  d["AU200708_08U"],d["AU200708_11U"],d["AU200708_15U"],d["AU200708_20U"],
                  d["AU200708_22U"],d["AU200708_23U"],d["AU200809_02U"],d["AU200809_03U"],
                  d["AU200809_08U"],d["AU200809_10U"],d["AU200809_18U"],d["AU200910_01U"],
                  d["AU200910_06U"],d["AU200910_12U"],d["AU200910_13U"],d["AU201011_02U"],
                  d["AU201011_09U"],d["AU201011_12U"],d["AU201011_16U"],d["AU201011_17U"],
                  d["AU201011_29U"],d["AU201112_01U"],d["AU201112_07U"],d["AU201112_11U"],
                  d["AU201112_15U"],d["AU201112_16U"],d["AU201213_04U"],d["AU201213_05U"],
                  d["AU201213_10U"],d["AU201213_17U"],d["AU201314_01U"],d["AU201314_03U"],
                  d["AU201314_04U"],d["AU201314_09U"],d["AU201314_14U"],d["AU201314_16U"],
                  d["AU201415_04U"],d["AU201415_16U"],d["AU201415_19U"],d["AU201415_21U"],
                  d["AU201516_08U"],d["AU201516_09U"],d["AU201617_07U"],d["AU201617_20U"],
                  d["AU201617_23U"],d["AU201617_26U"],d["AU201617_29U"],d["AU201617_30U"],
                  d["AU201718_02U"],d["AU201718_03U"],d["AU201718_06U"],d["AU201718_17U"],
                  d["AU201718_20U"],d["AU201819_05U"],d["AU201819_12U"],d["AU201819_17U"],
                  d["AU201819_19U"],d["AU201819_21U"],d["AU201819_25U"],d["AU201920_03U"],
                  d["AU201920_05U"],d["AU201920_08U"],d["AU202021_07U"],d["AU202021_10U"],
                  d["AU202021_15U"],d["AU202021_22U"],d["AU202021_23U"],d["AU202122_02U"],
                  d["AU202122_05U"],d["AU202122_22U"],d["AU202122_23U"],d["AU202122_27U"],
                  d["AU202122_28U"],d["AU202122_36U"],d["AU202223_01U"],d["AU202223_05U"],
                  d["AU202223_13U"],d["AU202223_21U"],d["AU202223_23U"],d["AU202324_04U"],
                  d["AU202324_08U"],d["AU202324_11U"])

df_west = [AU199697_14U,AU199899_05U,AU199900_13U,AU200203_02U,AU200203_03U,AU200203_04U,
           AU200203_06U,AU200203_07U,AU200304_01U,AU200304_03U,AU200304_05U,AU200304_07U,
           AU200304_08U,AU200304_11U,AU200405_01U,AU200405_02U,AU200405_03U,AU200405_04U,
           AU200405_07U,AU200405_08U,AU200405_09U,AU200506_01U,AU200506_05U,AU200506_06U,
           AU200506_14U,AU200506_16U,AU200506_20U,AU200607_11U,AU200607_12U,AU200607_13U,
           AU200708_01U,AU200708_06U,AU200708_08U,AU200708_11U,AU200708_15U,AU200708_20U,
           AU200708_22U,AU200708_23U,AU200809_02U,AU200809_03U,AU200809_08U,AU200809_10U,
           AU200809_18U,AU200910_01U,AU200910_06U,AU200910_12U,AU200910_13U,AU201011_02U,
           AU201011_09U,AU201011_12U,AU201011_16U,AU201011_17U,AU201011_29U,AU201112_01U,
           AU201112_07U,AU201112_11U,AU201112_15U,AU201112_16U,AU201213_04U,AU201213_05U,
           AU201213_10U,AU201213_17U,AU201314_01U,AU201314_03U,AU201314_04U,AU201314_09U,
           AU201314_14U,AU201314_16U,AU201415_04U,AU201415_16U,AU201415_19U,AU201415_21U,
           AU201516_08U,AU201516_09U,AU201617_07U,AU201617_20U,AU201617_23U,AU201617_26U,
           AU201617_29U,AU201617_30U,AU201718_02U,AU201718_03U,AU201718_06U,AU201718_17U,
           AU201718_20U,AU201819_05U,AU201819_12U,AU201819_17U,AU201819_19U,AU201819_21U,
           AU201819_25U,AU201920_03U,AU201920_05U,AU201920_08U,AU202021_07U,AU202021_10U,
           AU202021_15U,AU202021_22U,AU202021_23U,AU202122_02U,AU202122_05U,AU202122_22U,
           AU202122_23U,AU202122_27U,AU202122_28U,AU202122_36U,AU202223_01U,AU202223_05U,
           AU202223_13U,AU202223_21U,AU202223_23U,AU202324_04U,AU202324_08U,AU202324_11U]
# East region only
(AU200001_06U,AU200203_01U,AU200203_06U,AU200304_02U,AU200304_04U,AU200304_09U,
 AU200405_05U,AU200405_07U,AU200405_10U,AU200506_08U,AU200506_10U,AU200506_15U,
 AU200506_17U,AU200506_22U,AU200607_05U,AU200607_16U,AU200708_03U,AU200708_08U,
 AU200809_06U,AU200809_17U,AU200809_23U,AU200910_07U,AU200910_09U,AU200910_11U,
 AU201011_10U,AU201011_11U,AU201011_14U,AU201011_17U,AU201112_04U,AU201112_12U,
 AU201213_03U,AU201213_13U,AU201213_14U,AU201213_18U,AU201314_01U,AU201314_07U,
 AU201314_10U,AU201314_15U,AU201415_13U,AU201415_14U,AU201415_17U,AU201415_24U,
 AU201516_10U,AU201617_19U,AU201617_24U,AU201718_20U,AU201718_22U,AU201718_24U,
 AU201819_04U,AU201819_07U,AU201819_14U,AU201819_20U,AU201819_26U,AU201920_06U,
 AU201920_07U,AU201920_10U,AU201920_12U,AU202021_09U,AU202021_11U,AU202021_17U,
 AU202122_07U,AU202122_08U,AU202122_10U,AU202122_18U,AU202223_14U,AU202324_02U,
 AU202324_05U,AU202324_09U,
 AU202324_13U) = (d["AU200001_06U"],d["AU200203_01U"],d["AU200203_06U"],d["AU200304_02U"],
                  d["AU200304_04U"],d["AU200304_09U"],d["AU200405_05U"],d["AU200405_07U"],
                  d["AU200405_10U"],d["AU200506_08U"],d["AU200506_10U"],d["AU200506_15U"],
                  d["AU200506_17U"],d["AU200506_22U"],d["AU200607_05U"],d["AU200607_16U"],
                  d["AU200708_03U"],d["AU200708_08U"],d["AU200809_06U"],d["AU200809_17U"],
                  d["AU200809_23U"],d["AU200910_07U"],d["AU200910_09U"],d["AU200910_11U"],
                  d["AU201011_10U"],d["AU201011_11U"],d["AU201011_14U"],d["AU201011_17U"],
                  d["AU201112_04U"],d["AU201112_12U"],d["AU201213_03U"],d["AU201213_13U"],
                  d["AU201213_14U"],d["AU201213_18U"],d["AU201314_01U"],d["AU201314_07U"],
                  d["AU201314_10U"],d["AU201314_15U"],d["AU201415_13U"],d["AU201415_14U"],
                  d["AU201415_17U"],d["AU201415_24U"],d["AU201516_10U"],d["AU201617_19U"],
                  d["AU201617_24U"],d["AU201718_20U"],d["AU201718_22U"],d["AU201718_24U"],
                  d["AU201819_04U"],d["AU201819_07U"],d["AU201819_14U"],d["AU201819_20U"],
                  d["AU201819_26U"],d["AU201920_06U"],d["AU201920_07U"],d["AU201920_10U"],
                  d["AU201920_12U"],d["AU202021_09U"],d["AU202021_11U"],d["AU202021_17U"],
                  d["AU202122_07U"],d["AU202122_08U"],d["AU202122_10U"],d["AU202122_18U"],
                  d["AU202223_14U"],d["AU202324_02U"],d["AU202324_05U"],d["AU202324_09U"],
                  d["AU202324_13U"])
df_east = [AU200001_06U,AU200203_01U,AU200203_06U,AU200304_02U,AU200304_04U,AU200304_09U,
           AU200405_05U,AU200405_07U,AU200405_10U,AU200506_08U,AU200506_10U,AU200506_15U,
           AU200506_17U,AU200506_22U,AU200607_05U,AU200607_16U,AU200708_03U,AU200708_08U,
           AU200809_06U,AU200809_17U,AU200809_23U,AU200910_07U,AU200910_09U,AU200910_11U,
           AU201011_10U,AU201011_11U,AU201011_14U,AU201011_17U,AU201112_04U,AU201112_12U,
           AU201213_03U,AU201213_13U,AU201213_14U,AU201213_18U,AU201314_01U,AU201314_07U,
           AU201314_10U,AU201314_15U,AU201415_13U,AU201415_14U,AU201415_17U,AU201415_24U,
           AU201516_10U,AU201617_19U,AU201617_24U,AU201718_20U,AU201718_22U,AU201718_24U,
           AU201819_04U,AU201819_07U,AU201819_14U,AU201819_20U,AU201819_26U,AU201920_06U,
           AU201920_07U,AU201920_10U,AU201920_12U,AU202021_09U,AU202021_11U,AU202021_17U,
           AU202122_07U,AU202122_08U,AU202122_10U,AU202122_18U,AU202223_14U,AU202324_02U,
           AU202324_05U,AU202324_09U,AU202324_13U]
################################################################################

### DATA MODELING

In [3]:
# for reproducibility
torch.manual_seed(1)
np.random.seed(1)

def generate_torch_kwargs():
    torch.manual_seed(1)
    np.random.seed(1)
    
    # run torch models on CPU, and disable progress bars for all model stages except training.
    return {
        "pl_trainer_kwargs": {
            "accelerator": "cpu",
            "callbacks": [TFMProgressBar(enable_train_bar_only=True)],
        }
    }

def modelfitting(tscols=['LAT',
                         'LON',
                         'CENTRAL_PRES',
                         'ENV_PRES',
                         'MN_RADIUS_GF_SECNE',
                         'MN_RADIUS_GF_SECSE',
                         'MN_RADIUS_GF_SECSW',
                         'MN_RADIUS_GF_SECNW'],
                 df_list_of_cyclones=df_all,
                 covariates=None):
    """
    N-BEATS model fitting.
    Take a name list of target features tscols.
    Default tscols = ['LAT',
          'LON',
          'CENTRAL_PRES',
          'ENV_PRES',
          'MN_RADIUS_GF_SECNE',
          'MN_RADIUS_GF_SECSE',
          'MN_RADIUS_GF_SECSW',
          'MN_RADIUS_GF_SECNW']
    Take a df list of validating cyclones df_list_of_cyclones, default is df_all.
    Return a dataframe for model evaluation report.
    Return a dataframe for geoplot if 'LON' and 'LAT' are both in target features.
    """

    # for reproducibility
    torch.manual_seed(1)
    np.random.seed(1)

    # Storage for evaluation report
    bt_act_df_list = []
    bt_fc_df_list = []
    sc_bt_act_df_list = []
    sc_bt_fc_df_list = []
    bt_error_list = []

    # Flag used to monitor whether previous tracking data has been stored
    flag_geo_plot_data = False

    # Temporary variable to limit number of cyclones used for validation
    max_cyclones = 5
    cyclones_so_far = 1

    for cyclone in df_list_of_cyclones:
        cyclone_name = str(cyclone.iat[0, 1])
        print("Length = " + str(len(cyclone)))
        print("Cyclone number = " + str(cyclones_so_far))
        if cyclones_so_far > max_cyclones:
            break
        print("Cyclone ID: " + str(cyclone.iat[0, 1]))
        # Check length of this cyclone data
        if len(cyclone) > 13:
            cycdf = d.get(cyclone_name)

            # Validation Target series
            validation_ts = TimeSeries.from_dataframe(cycdf, "TM", tscols)
            # Scaling
            validation_scaler = Scaler()
            # Scale Validation data before splitting
            validation_data_scaled = validation_scaler.fit_transform(validation_ts)
            # Keep the last 8 obs for validation of prediction
            (train_validation_data_scaled,
             actual_validation_data_scaled) = (validation_data_scaled[:-8],
                                               validation_data_scaled[-8:])

            # Create temporary copy of dictionary
            dd = {key: value[:] for key, value in d.items()}
            # Remove validation cyclone from temp dictionary
            dd.pop(cyclone.iat[0, 1])

            # Only add cyclones with sufficient number of observations for model to work
            cyclones = []
            for ea in dd:
                this_cyclone = ea
                if len(dd.get(this_cyclone)) > 11:
                    result = dd.get(this_cyclone)
                    cyclones.append(result)

            # Handle target series

            # Convert all target series in list to Time Series objects
            ts_cyclones = []
            for ea in cyclones:
                result = TimeSeries.from_dataframe(ea, "TM", tscols)
                ts_cyclones.append(result)

            # Scale all series in list
            sc_ts_cyclone_list = []
            for ea in ts_cyclones:
                new_scaler = Scaler()
                result = new_scaler.fit_transform(ea)
                sc_ts_cyclone_list.append(result)

            # Handle covariates
            if covariates is not None:

                # Validation Covariate series
                val_cov_ts = TimeSeries.from_dataframe(cycdf, "TM", covariates)
                val_cov_scaler = Scaler()
                # Scale covariate series
                val_cov_scaled = val_cov_scaler.fit_transform(val_cov_ts)

                # Training covariate series

                # Convert all covariate series in list to Time Series objects
                past_cov = []
                for ea in cyclones:
                    result = TimeSeries.from_dataframe(ea, "TM", covariates)
                    past_cov.append(result)

                # Scale all series in list
                sc_past_cov = []
                for ea in past_cov:
                    new_scaler = Scaler()
                    result = new_scaler.fit_transform(ea)
                    sc_past_cov.append(result)
            else:
                sc_past_cov = None
                val_cov_scaled = None

            # Train model
            model_NBEATS = NBEATSModel(
                input_chunk_length=6,
                output_chunk_length=4,
                n_epochs=30,
                random_state=0,
                **generate_torch_kwargs())

            model_NBEATS.fit(sc_ts_cyclone_list,
                             past_covariates=sc_past_cov)

            # BACKTEST

            # HISTORICAL FORECASTS for SCALED DATA UNIVARIATE COMPONENT
            sc_bt_fc = model_NBEATS.historical_forecasts(validation_data_scaled,
                                                         forecast_horizon=8,
                                                         retrain=False,
                                                         last_points_only=False,
                                                         past_covariates=val_cov_scaled)

            # Create list of forecasts
            sc_bt_fc_list = [np.zeros(len(tscols))]
            for i in range(0, len(sc_bt_fc)):
                a = sc_bt_fc[i].values()
                sc_bt_fc_list = np.concatenate([sc_bt_fc_list, a])
            sc_bt_fc_list = sc_bt_fc_list[1:]

            # Create list of run no and cyc id
            run_no = [cyclones_so_far] * 8 * len(sc_bt_fc)
            cyc_id = [cyclone_name] * 8 * len(sc_bt_fc)

            # Create list of corresponding actual values
            sc_bt_act_list = [np.zeros(len(tscols))]
            for i in range(0, len(sc_bt_fc)):
                start = 6 + i
                end = 6 + i + 8
                a = validation_data_scaled[start:end].values()
                sc_bt_act_list = np.concatenate([sc_bt_act_list, a])
            sc_bt_act_list = sc_bt_act_list[1:]

            # Create list of index of forecasts
            sc_obs = np.array(range(1, 9))
            for i in range(len(sc_bt_fc) - 1):
                b = np.array(range(1, 9))
                sc_obs = np.concatenate([sc_obs, b])

            # Create df for actual for output
            sc_bt_act_df = pd.DataFrame(index=range(len(sc_bt_act_list)),
                                        data=sc_bt_act_list,
                                        columns=tscols)
            sc_bt_act_df['Obs'] = sc_obs
            sc_bt_act_df['run_no'] = run_no
            sc_bt_act_df['cyc_id'] = cyc_id

            # Create df for forecast for output
            sc_bt_fc_df = pd.DataFrame(index=range(len(sc_bt_fc_list)),
                                       data=sc_bt_fc_list,
                                       columns=tscols)
            sc_bt_fc_df['Obs'] = sc_obs
            sc_bt_fc_df['run_no'] = run_no
            sc_bt_fc_df['cyc_id'] = cyc_id

            # HISTORICAL FORECASTS for ORIGINAL DATA UNIVARIATE COMPONENT

            # Inverse scaling scaled historical forecasts
            bt_fc = []
            for ts in sc_bt_fc:
                inv_sc_bt = validation_scaler.inverse_transform(ts)
                bt_fc.append(inv_sc_bt)

                # Create list of forecasts
            bt_fc_list = [np.zeros(len(tscols))]
            for i in range(0, len(bt_fc)):
                a = bt_fc[i].values()
                bt_fc_list = np.concatenate([bt_fc_list, a])
            bt_fc_list = bt_fc_list[1:]

            # Create list of corresponding actual values
            bt_act_list = [np.zeros(len(tscols))]
            for i in range(0, len(bt_fc)):
                start = 6 + i
                end = 6 + i + 8
                a = validation_ts[start:end].values()
                bt_act_list = np.concatenate([bt_act_list, a])
            bt_act_list = bt_act_list[1:]

            # Create list of index of forecasts
            obs = np.array(range(1, 9))
            for i in range(len(bt_fc) - 1):
                b = np.array(range(1, 9))
                obs = np.concatenate([obs, b])

            # Create df for actual for output
            bt_act_df = pd.DataFrame(index=range(len(bt_act_list)),
                                     data=bt_act_list,
                                     columns=tscols)
            bt_act_df['Obs'] = obs
            bt_act_df['run_no'] = run_no
            bt_act_df['cyc_id'] = cyc_id

            # Create df for forecast for output
            bt_fc_df = pd.DataFrame(index=range(len(bt_fc_list)),
                                    data=bt_fc_list,
                                    columns=tscols)
            bt_fc_df['Obs'] = obs
            bt_fc_df['run_no'] = run_no
            bt_fc_df['cyc_id'] = cyc_id

            # BACK TEST FOR MODEL GENERAL PERFORMANCE
            bt_error = model_NBEATS.backtest(validation_data_scaled,
                                             metric=[mae, rmse, mse],
                                             forecast_horizon=8,
                                             retrain=False,
                                             past_covariates=val_cov_scaled)
            bt_dict = {'run_no': cyclones_so_far,
                       'cyc_id': cyclone_name,
                       'MAE': bt_error[0],
                       'RMSE': bt_error[1],
                       'MSE': bt_error[2]}
            bt_error_df = pd.DataFrame(bt_dict, index=[cyclones_so_far - 1])

            # Append to storage list
            bt_act_df_list.append(bt_act_df)
            bt_fc_df_list.append(bt_fc_df)
            sc_bt_act_df_list.append(sc_bt_act_df)
            sc_bt_fc_df_list.append(sc_bt_fc_df)
            bt_error_list.append(bt_error_df)

            # Run prediction train_validation_data_scaled for plotting
            sc_forecast_NBEATS = model_NBEATS.predict(n=8,
                                                      series=train_validation_data_scaled,
                                                      past_covariates=val_cov_scaled)

            # Reverse scaling (at n=8 for plotting)
            forecast_NBEATS = validation_scaler.inverse_transform(sc_forecast_NBEATS)
            actual_validation = validation_scaler.inverse_transform(actual_validation_data_scaled)

            if 'LON' in tscols and 'LAT' in tscols:
                for idx, component in enumerate(tscols):
                    if component == 'LAT':
                        variable1_pred = forecast_NBEATS.univariate_component(idx)
                        variable1_act = actual_validation.univariate_component(idx)
                        variable1_past = validation_ts[:-8].univariate_component(idx)

                    if component == 'LON':
                        variable2_pred = forecast_NBEATS.univariate_component(idx)
                        variable2_act = actual_validation.univariate_component(idx)
                        variable2_past = validation_ts[:-8].univariate_component(idx)

                variable1_pred_values = variable1_pred.values()
                variable2_pred_values = variable2_pred.values()
                variable1_act_values = variable1_act.values()
                variable2_act_values = variable2_act.values()
                variable1_past_values = variable1_past.values()
                variable2_past_values = variable2_past.values()

                # Concatenate values of the past, actual & prediction
                #  data to make plots continuous
                variable1_existing_values = np.concatenate([variable1_past_values,
                                                            variable1_act_values])
                variable2_existing_values = np.concatenate([variable2_past_values,
                                                            variable2_act_values])
                variable1_pred_values = np.concatenate([variable1_past_values,
                                                        variable1_pred_values])
                variable2_pred_values = np.concatenate([variable2_past_values,
                                                        variable2_pred_values])

                # Create datetime variables
                TM_act = variable1_act.time_index
                TM_past = variable1_past.time_index
                TM_exist = np.concatenate([TM_past, TM_act])

                # Create dataframe for plotting with geopandas
                geo_plot_all = pd.DataFrame(index=range(len(variable1_existing_values)))
                geo_plot_all['LAT'] = variable1_existing_values
                geo_plot_all['LON'] = variable2_existing_values
                geo_plot_all['TM'] = TM_exist
                geo_plot_all['DISTURBANCE_ID'] = str(cyclone_name + "-Real")
                geo_plot_pred = pd.DataFrame(index=range(len(variable1_pred_values)))
                geo_plot_pred['LAT'] = variable1_pred_values
                geo_plot_pred['LON'] = variable2_pred_values
                geo_plot_pred['TM'] = TM_exist
                geo_plot_pred['DISTURBANCE_ID'] = str(cyclone_name + "-Predicted")
                if not flag_geo_plot_data:
                    geo_plot_data = pd.concat([geo_plot_pred,
                                               geo_plot_all],
                                              ignore_index=True)
                    flag_geo_plot_data = True
                else:
                    geo_plot_data = pd.concat([geo_plot_pred,
                                               geo_plot_all,
                                               geo_plot_data],
                                              ignore_index=True)

            cyclones_so_far += 1
    bt_out = [bt_act_df_list,
              bt_fc_df_list,
              sc_bt_act_df_list,
              sc_bt_fc_df_list,
              bt_error_list]
    for m, n in enumerate(bt_out):
        bt_out[m] = pd.concat(n)
    if 'LON' in tscols and 'LAT' in tscols:
        return [bt_out,
                tscols,
                len(cyclones),
                cyclones_so_far - 1,
                covariates,
                geo_plot_data]
    else:
        return [bt_out,
                tscols,
                len(cyclones),
                cyclones_so_far - 1,
                covariates]

# Self-defined function for Geoplotting
def geoplot(geo_plot_data, plotname='trajectory_plot.html'):
    """
    Mapping cyclone path.
    Take in a data frame containing data for plotting.
    Allow input of plotname, default plotname = 'trajectory_plot.html'.
    """
    # Set CRS to WGS84 coordinate system
    crs = {'init': 'epsg:4326'}
    # Convert Longitude and Latitude to Points
    geometry = [Point(xy) for xy in zip(geo_plot_data['LON'], geo_plot_data['LAT'])]
    geo_data = gpd.GeoDataFrame(geo_plot_data, geometry=geometry)

    # Create trajectories
    geo_data['Time'] = pd.to_datetime(geo_data['TM'],
                                      format='%d/%m/%Y %H:%M',
                                      errors='coerce')
    geo_data = geo_data.set_index('Time')
    # Specify minimum length of trajectories
    minimum_length = 0

    # Create Trajectory Collection
    traj_collection = mpd.TrajectoryCollection(geo_data,
                                               'DISTURBANCE_ID',
                                               min_length=minimum_length)

    # Create DataFrame to store trajectory coordinates
    trajectory_df = pd.DataFrame(columns=['DISTURBANCE_ID', 'LON', 'LAT'])

    # For each trajectory in the TrajectoryCollection
    #  Extract coordinates and add to the trajectory DataFrame
    for traj in traj_collection:
        coords = traj.to_linestring().coords[:]
        traj_id = traj.id
        for LON, LAT in coords:
            trajectory_df = pd.concat([trajectory_df,
                                       pd.DataFrame([{'DISTURBANCE_ID': traj_id,
                                                      'LON': LON,
                                                      'LAT': LAT}])],
                                      ignore_index=True)
    # Create plot
    fig = px.line_mapbox(trajectory_df,
                         lat='LAT', lon='LON',
                         color='DISTURBANCE_ID',
                         hover_name='DISTURBANCE_ID',
                         zoom=4)
    # Set layout options for map
    fig.update_layout(mapbox_style="carto-positron",
                      mapbox_zoom=4,
                      mapbox_center={"lat": trajectory_df['LAT'].mean(),
                                     "lon": trajectory_df['LON'].mean()})
    # Write html file containing interactive plot
    fig.write_html(plotname)
    print(f"A geoplot named '{plotname}' was created.")

def unicom_rep(scaled=False):
    """
    Univariate component report.
    Default scaled = False to report on original series.
    Change scaled = True to report on scaled series.
    """
    tscols = target_col

    print('\n')
    print('#' * 50)
    print('UNIVARIATE COMPONENT REPORT\n')
    print('#' * 50)
    if scaled == True:
        bt_act_df = sc_bt_act
        bt_fc_df = sc_bt_fc
        print('Table of Scaled Error for Each Component.')
        print('\nError Scores: scaled between 0 to 1')
    else:
        bt_act_df = bt_act
        bt_fc_df = bt_fc
        print('Table of Error Scores for Each Component.')
    print('#' * 50)
    print('\n')
    for cycid in bt_fc_df['cyc_id'].unique():
        for comp in tscols:
            print('\nCyclone ID: ', cycid)
            print('Cyclone Name:', compare_dict[cycid])
            print('Component: ', comp)

            bt_rmse = []
            bt_mse = []
            bt_mae = []
            bt_obs = []
            bt_len = []
            for g in bt_fc_df['Obs'].unique():
                mask_act = bt_act_df.loc[(bt_act_df['Obs'] == g) &
                                         (bt_act_df['cyc_id'] == cycid), comp]
                mask_fc = bt_fc_df.loc[(bt_fc_df['Obs'] == g) &
                                       (bt_fc_df['cyc_id'] == cycid), comp]

                rmse = mean_squared_error(mask_act, mask_fc, squared=False)
                mse = mean_squared_error(mask_act, mask_fc, squared=True)
                mae = mean_absolute_error(mask_act, mask_fc)

                bt_rmse.append(rmse)
                bt_mse.append(mse)
                bt_mae.append(mae)
                bt_obs.append(g)
                bt_len.append(len(mask_fc))

            bt_rep = pd.DataFrame(list(zip(bt_len, bt_mae, bt_rmse, bt_mse)),
                                  columns=['BT_sets', 'MAE', 'RMSE', 'MSE'])

            bt_rep = bt_rep.transpose().round(2)
            headers = ['6h\n(n=1)', '12h\n(n=2)', '18h\n(n=3)', '24h\n(n=4)',
                       '30h\n(n=5)', '36h\n(n=6)', '42h\n(n=7)', '48h\n(n=8)']

            if scaled == False:
                if comp in ['LAT', 'LON']:
                    nm = bt_rep.copy()
                    nm.loc[['MAE', 'RMSE', 'MSE']] = nm.loc[['MAE', 'RMSE', 'MSE']] * 60
                    print('\nError Units: nautical miles (nm)')
                    print(tabulate(nm, headers=headers, tablefmt='psql'))
                    print('\nError Units: decimal degrees')

                elif comp in ['MN_RADIUS_GF_SECNE',
                              'MN_RADIUS_GF_SECSE',
                              'MN_RADIUS_GF_SECSW',
                              'MN_RADIUS_GF_SECNW']:
                    print('\nError Units: kilometres')

                elif comp in ['ENV_PRES', 'CENTRAL_PRES']:
                    print('\nError Units: hectopascals')

            elif scaled == True:
                print('\nError Units: scaled between 0 and 1')

            print(tabulate(bt_rep, headers=headers, tablefmt='psql'))
            print('\n')
            print('#' * 50)
            print('\n')

def eval_rep(scaled=True):
    """
    Generate MODEL EVALUATION REPORT FOR ALL COMPONENTS
    """
    tscols = target_col
    traincycount = traincycount_fit
    valcycount = valcycount_fit
    bt_error = bt_error_fit
    covariates = covariates_fit

    print('#' * 50)
    print('MODEL EVALUATION REPORT FOR ALL COMPONENTS')
    print('#' * 50)
    print('\nNumber of fitted target features: ' + str(len(tscols)))
    print('List of fitted target features: ')
    print([i for i in tscols])
    print('List of fitted covariate features:')
    if covariates is not None:
        print([i for i in covariates])
    else:
        print('No covariate features.')
    print('Number of training cyclones used for each run: ' + str(traincycount))
    print('Number of validating cyclones used for each run: 1')
    print('Number of validating runs: ' + str(valcycount))
    print('Prediction at step n ahead: n= ', [i for i in range(1, 9)])

    print('\n' + '#' * 50)

    if scaled == True:
        bt_act_df = sc_bt_act
        bt_fc_df = sc_bt_fc
        print('Table of Scaled Error for Model Performance.')
    else:
        bt_act_df = bt_act
        bt_fc_df = bt_fc
        print('Table of Error Scores for Model Performance.')
    print('#' * 50)
    print('\n')

    for cycid in bt_fc_df['cyc_id'].unique():
        print('\nCyclone ID: ', cycid)
        print('Cyclone Name:', compare_dict[cycid])
        bt_rmse = []
        bt_mse = []
        bt_mae = []
        bt_obs = []
        bt_len = []
        for g in bt_fc_df['Obs'].unique():
            mask_act = bt_act_df.loc[(bt_act_df['Obs'] == g) &
                                     (bt_act_df['cyc_id'] == cycid), tscols]
            mask_fc = bt_fc_df.loc[(bt_fc_df['Obs'] == g) &
                                   (bt_fc_df['cyc_id'] == cycid), tscols]

            rmse = mean_squared_error(mask_act, mask_fc, squared=False)
            mse = mean_squared_error(mask_act, mask_fc, squared=True)
            mae = mean_absolute_error(mask_act, mask_fc)

            bt_rmse.append(rmse)
            bt_mse.append(mse)
            bt_mae.append(mae)
            bt_obs.append(g)
            bt_len.append(len(mask_fc))

        bt_rep = pd.DataFrame(list(zip(bt_len, bt_mae, bt_rmse, bt_mse)),
                              columns=['BT_sets', 'MAE', 'RMSE', 'MSE'])

        # Add Backtest error output
        mask_bt_error = bt_error.loc[bt_error['cyc_id'] == cycid]
        a = {'BT_sets': bt_len[0],
             'MAE': mask_bt_error['MAE'],
             'RMSE': mask_bt_error['RMSE'],
             'MSE': mask_bt_error['MSE']}
        bt_rep = pd.concat([bt_rep, pd.DataFrame(a)], ignore_index=True)

        bt_rep = bt_rep.round(2).transpose()

        bt_rep.columns = ['6h', '12h', '18h', '24h', '30h', '36h', '42h', '48h', 'Overall']
        bt_rep = bt_rep[['Overall'] + [x for x in bt_rep.columns if x != 'Overall']]

        headers = ['Overall', '6h\n(n=1)', '12h\n(n=2)', '18h\n(n=3)', '24h\n(n=4)',
                   '30h\n(n=5)', '36h\n(n=6)', '42h\n(n=7)', '48h\n(n=8)']

        if scaled == True:
            print('Error Units: scaled between 0 to 1')

        print(tabulate(bt_rep, headers=headers, tablefmt='psql'))
        print('\n')
        print('#' * 50)



In [4]:
def summary_rep(repname='sumrep.csv'):
    """
    SUMMARY REPORT FOR MODEL EVALUATION
    Output csv file mean errors of all validating cyclones
    """
    tscols = target_col
    traincycount = traincycount_fit
    valcycount = valcycount_fit
    bt_error = bt_error_fit
    covariates = covariates_fit
    bt_act_df = sc_bt_act
    bt_fc_df = sc_bt_fc
    
    print('#'*50)
    print('SUMMARY REPORT FOR MODEL EVALUATION')
    print('#'*50)
    print('\nNumber of fitted target features: '+ str(len(tscols)))
    print('List of fitted target features: ')
    print([i for i in tscols])
    print('List of fitted covariate features:')
    if covariates is not None:
        print([i for i in covariates])
    else:
        print('No covariate features.')
    print('\nValidating Cyclone ID(s): ')
    print([cycid for cycid in bt_fc_df['cyc_id'].unique()])
    print('Validating Cyclone Name(s):')
    print([compare_dict[cycid] for cycid in bt_fc_df['cyc_id'].unique()])
    print('\nNumber of training cyclones used for each run: ' + str(traincycount))
    print('Validating method: leave one out')
    print('Number of validating runs: ' + str(valcycount))
    print('Prediction at step n ahead: n= ', [i for i in range(1,9)])
    print('\n'+'#'*50)
    print(f'Summarry Table of Scaled Error after {valcycount} Validating Runs.')
    print('#'*50)    
    print('\n')
    

    bt_rmse = []
    bt_mse = []
    bt_mae = []
    bt_obs = []
    bt_len = []
    
    # Calculate error scores
    for g in bt_fc_df['Obs'].unique():

        mask_act = bt_act_df.loc[(bt_act_df['Obs']==g),tscols]
        mask_fc = bt_fc_df.loc[(bt_fc_df['Obs']==g),tscols]

        rmse = mean_squared_error(mask_act, mask_fc, squared=False)
        mse = mean_squared_error(mask_act, mask_fc, squared=True)
        mae = mean_absolute_error(mask_act, mask_fc)

        bt_rmse.append(rmse)
        bt_mse.append(mse)
        bt_mae.append(mae)
        bt_obs.append(g)
        bt_len.append(len(mask_fc))


    bt_rep = pd.DataFrame(list(zip(bt_len,bt_mae,bt_rmse,bt_mse)),columns=['BT_sets','MAE', 'RMSE','MSE'])
    
#     # Save to csv before transpose
#     bt_rep.to_csv(f'{repname}.csv',index=False)
    
    bt_rep=bt_rep.round(2).transpose()

    bt_rep.columns = ['6h','12h','18h','24h','30h','36h','42h','48h']

    headers = ['6h\n(n=1)', '12h\n(n=2)', '18h\n(n=3)', '24h\n(n=4)', 
             '30h\n(n=5)', '36h\n(n=6)', '42h\n(n=7)', '48h\n(n=8)']

    print('Error Units: scaled between 0 to 1')   
    print(tabulate(bt_rep, headers = headers, tablefmt = 'psql'))
    print('\n')
    print('#'*50)

In [5]:
def bt_set_errors (modelname='fit'):
    bt_rmse = []
    bt_mse = []
    bt_mae = []
    bt_set_no = []


    tscols = target_col
    bt_act_df = sc_bt_act
    bt_fc_df = sc_bt_fc

    
    bt_count = int(len(bt_fc_df)/(bt_fc_df['Obs'].nunique()))
    
    bt_id = []
    for i in range(1,bt_count+1):
        a = [i]*bt_fc_df['Obs'].nunique()
        bt_id = bt_id + a

    bt_fc_df['bt_id'] = bt_id
    bt_act_df['bt_id'] = bt_id

    # Calculate error scores for each back test set
    for g in bt_fc_df['bt_id'].unique():
        mask_act = bt_act_df.loc[(bt_act_df['bt_id']==g),tscols]
        mask_fc = bt_fc_df.loc[(bt_fc_df['bt_id']==g),tscols]
        rmse = mean_squared_error(mask_act, mask_fc, squared=False)
        mse = mean_squared_error(mask_act, mask_fc, squared=True)
        mae = mean_absolute_error(mask_act, mask_fc)
        bt_rmse.append(rmse)
        bt_mse.append(mse)
        bt_mae.append(mae)
        bt_set_no.append(g)

    bt_set_error = pd.DataFrame(list(zip(bt_set_no,bt_mae,bt_rmse,bt_mse)),columns=['BT_ID','MAE', 'RMSE','MSE'])
    bt_set_error.to_csv(f'bt_set_error_{modelname}.csv', index=False)

In [6]:
def bt_csv (modelname='fit'):
    """
    save backtest outputs to 4 x csv files.
    """
    bt_act.to_csv(f'bt_act_{modelname}.csv',index=False)
    bt_fc.to_csv(f'bt_fc_{modelname}.csv',index=False)
    sc_bt_act.to_csv(f'sc_bt_act_{modelname}.csv',index=False)
    sc_bt_fc.to_csv(f'sc_bt_fc_{modelname}.csv',index=False)

### Model Fitting and Evaluation

In [7]:
# name list of target features
cycpath = ['LAT',
          'LON']

# name list of covariate features
pres = ['CENTRAL_PRES',
          'ENV_PRES']

r34s = ['MN_RADIUS_GF_SECNE',
        'MN_RADIUS_GF_SECSE',
        'MN_RADIUS_GF_SECSW',
        'MN_RADIUS_GF_SECNW']

# list of comparison cyclones from BOM for validating
compare = [AU202021_11U, AU202021_15U, AU202021_17U, AU202021_22U]
compare_id = []
compare_name = []
for cyc in compare:
    cycid = cyc['DISTURBANCE_ID'].unique().tolist()
    compare_id += cycid
    
for cyc in compare:
    name = cyc['NAME'].unique().tolist()
    compare_name += name

compare_dict = dict(zip(compare_id,compare_name))
# full excel files compare = [AU202021_02U, AU202021_07U, AU202021_09U, AU202021_11U, AU202021_15U, AU202021_17U, AU202021_22U]

#### 1. Modeling Cyclone Path Only

In [8]:
# Fitting cycpath
path_nocov = modelfitting(tscols=cycpath,df_list_of_cyclones=compare)

Length = 14
Cyclone number = 1
Cyclone ID: AU202021_11U
Epoch 29: 100%|██████████████████████████████████████████████████| 131/131 [00:16<00:00,  8.00it/s, train_loss=0.00102]
Length = 33
Cyclone number = 2
Cyclone ID: AU202021_15U
Epoch 29: 100%|██████████████████████████████████████████████████| 131/131 [00:16<00:00,  7.78it/s, train_loss=0.00195]
Length = 18
Cyclone number = 3
Cyclone ID: AU202021_17U
Epoch 29: 100%|██████████████████████████████████████████████████| 131/131 [00:16<00:00,  7.87it/s, train_loss=0.00107]
Length = 30
Cyclone number = 4
Cyclone ID: AU202021_22U
Epoch 29: 100%|██████████████████████████████████████████████████| 131/131 [00:16<00:00,  7.97it/s, train_loss=0.00181]


In [9]:
# IMPORTANT: only need to change model name in [] in for loop.
# Don't change names of variables on the left hand side.

for fittedmodel in [path_nocov]:
    ### DON'T CHANGE BELOW CODE
    
    # Assign backtest output of fitted model for reports
    bt_act = fittedmodel[0][0]
    bt_fc = fittedmodel[0][1]
    sc_bt_act = fittedmodel[0][2]
    sc_bt_fc = fittedmodel[0][3]
    bt_error_fit = fittedmodel[0][4]

    # Assign other outputs from fitted model for reports
    target_col = fittedmodel[1]
    traincycount_fit = fittedmodel[2]
    valcycount_fit = fittedmodel[3]
    covariates_fit = fittedmodel[4]
    
    ### DON'T CHANGE ABOVE CODE

In [10]:
# save back test outputs to csv files
bt_csv(modelname='path_nocov')

In [11]:
# calculate back test set error and save to csv files
bt_set_errors(modelname='path_nocov')

In [12]:
# summary report
summary_rep()

##################################################
SUMMARY REPORT FOR MODEL EVALUATION
##################################################

Number of fitted target features: 2
List of fitted target features: 
['LAT', 'LON']
List of fitted covariate features:
No covariate features.

Validating Cyclone ID(s): 
['AU202021_11U', 'AU202021_15U', 'AU202021_17U', 'AU202021_22U']
Validating Cyclone Name(s):
['Lucas', 'Marian', 'Niran', 'Seroja']

Number of training cyclones used for each run: 93
Validating method: leave one out
Number of validating runs: 4
Prediction at step n ahead: n=  [1, 2, 3, 4, 5, 6, 7, 8]

##################################################
Summarry Table of Scaled Error after 4 Validating Runs.
##################################################


Error Units: scaled between 0 to 1
+---------+---------+---------+---------+---------+---------+---------+---------+---------+
|         |      6h |     12h |     18h |     24h |     30h |     36h |     42h |     48h |
|        

In [13]:
# Geoplot for cycpath
if len(path_nocov)==6:
    geoplot(geo_plot_data=path_nocov[-1],plotname='path_nocov.html')
else:
    print('No data for geoplot.')

A geoplot named 'path_nocov.html' was created.


#### 2. Modeling Cyclone Path with Covariates 'CENTRAL_PRES',  'ENV_PRES'

In [14]:
# Fitting cycpath with cov 'CENTRAL_PRES', 'ENV_PRES'
path_cov_pres = modelfitting(tscols=cycpath,df_list_of_cyclones=compare, covariates=pres)

Length = 14
Cyclone number = 1
Cyclone ID: AU202021_11U
Epoch 29: 100%|██████████████████████████████████████████████████| 131/131 [00:17<00:00,  7.58it/s, train_loss=0.00047]
Length = 33
Cyclone number = 2
Cyclone ID: AU202021_15U
Epoch 29: 100%|█████████████████████████████████████████████████| 131/131 [00:16<00:00,  7.73it/s, train_loss=0.000742]
Length = 18
Cyclone number = 3
Cyclone ID: AU202021_17U
Epoch 29: 100%|██████████████████████████████████████████████████| 131/131 [00:16<00:00,  7.82it/s, train_loss=0.00057]
Length = 30
Cyclone number = 4
Cyclone ID: AU202021_22U
Epoch 29: 100%|█████████████████████████████████████████████████| 131/131 [00:17<00:00,  7.45it/s, train_loss=0.000938]


In [15]:
# IMPORTANT: only need to change model name in [] in 'for loop' for reports
# Don't change names of variables on the left hand side.

for fittedmodel in [path_cov_pres]:
    ### DON'T CHANGE BELOW CODE
    
    # Assign backtest output of fitted model for reports
    bt_act = fittedmodel[0][0]
    bt_fc = fittedmodel[0][1]
    sc_bt_act = fittedmodel[0][2]
    sc_bt_fc = fittedmodel[0][3]
    bt_error_fit = fittedmodel[0][4]

    # Assign other outputs from fitted model for reports
    target_col = fittedmodel[1]
    traincycount_fit = fittedmodel[2]
    valcycount_fit = fittedmodel[3]
    covariates_fit = fittedmodel[4]
    
    ### DON'T CHANGE ABOVE CODE

In [16]:
# save back test outputs to csv files
bt_csv(modelname='path_cov_pres')

In [17]:
# calculate back test set error and save to csv files
bt_set_errors(modelname='path_cov_pres')

In [18]:
# summary report
summary_rep()

##################################################
SUMMARY REPORT FOR MODEL EVALUATION
##################################################

Number of fitted target features: 2
List of fitted target features: 
['LAT', 'LON']
List of fitted covariate features:
['CENTRAL_PRES', 'ENV_PRES']

Validating Cyclone ID(s): 
['AU202021_11U', 'AU202021_15U', 'AU202021_17U', 'AU202021_22U']
Validating Cyclone Name(s):
['Lucas', 'Marian', 'Niran', 'Seroja']

Number of training cyclones used for each run: 93
Validating method: leave one out
Number of validating runs: 4
Prediction at step n ahead: n=  [1, 2, 3, 4, 5, 6, 7, 8]

##################################################
Summarry Table of Scaled Error after 4 Validating Runs.
##################################################


Error Units: scaled between 0 to 1
+---------+---------+---------+---------+---------+---------+---------+---------+---------+
|         |      6h |     12h |     18h |     24h |     30h |     36h |     42h |     48h |
|  

In [19]:
# Geoplot
if len(path_cov_pres)==6:
    geoplot(geo_plot_data=path_cov_pres[-1],plotname='path_cov_pres.html')
else:
    print('No data for geoplot.')

A geoplot named 'path_cov_pres.html' was created.


#### 3. Modeling Cyclone Path with Covariates R34s

In [20]:
# Fitting cycpath with cov R34s
path_cov_r34s = modelfitting(tscols=cycpath,df_list_of_cyclones=compare, covariates=r34s)

Length = 14
Cyclone number = 1
Cyclone ID: AU202021_11U
Epoch 29: 100%|█████████████████████████████████████████████████| 131/131 [00:17<00:00,  7.35it/s, train_loss=0.000931]
Length = 33
Cyclone number = 2
Cyclone ID: AU202021_15U
Epoch 29: 100%|██████████████████████████████████████████████████| 131/131 [00:18<00:00,  7.23it/s, train_loss=0.00045]
Length = 18
Cyclone number = 3
Cyclone ID: AU202021_17U
Epoch 29: 100%|██████████████████████████████████████████████████| 131/131 [00:17<00:00,  7.34it/s, train_loss=0.00189]
Length = 30
Cyclone number = 4
Cyclone ID: AU202021_22U
Epoch 29: 100%|██████████████████████████████████████████████████| 131/131 [00:18<00:00,  7.00it/s, train_loss=0.00091]


In [21]:
# IMPORTANT: only need to change model name in [] in 'for loop' for reports
# Don't change names of variables on the left hand side.

for fittedmodel in [path_cov_r34s]:
    
    ### DON'T CHANGE BELOW CODE
    
    # Assign backtest output of fitted model for reports
    bt_act = fittedmodel[0][0]
    bt_fc = fittedmodel[0][1]
    sc_bt_act = fittedmodel[0][2]
    sc_bt_fc = fittedmodel[0][3]
    bt_error_fit = fittedmodel[0][4]

    # Assign other outputs from fitted model for reports
    target_col = fittedmodel[1]
    traincycount_fit = fittedmodel[2]
    valcycount_fit = fittedmodel[3]
    covariates_fit = fittedmodel[4]
    
    ### DON'T CHANGE ABOVE CODE

In [22]:
# save back test outputs to csv files
bt_csv(modelname='path_cov_r34s')

In [23]:
# calculate back test set error and save to csv files
bt_set_errors(modelname='path_cov_r34s')

In [24]:
# summary report
summary_rep()

##################################################
SUMMARY REPORT FOR MODEL EVALUATION
##################################################

Number of fitted target features: 2
List of fitted target features: 
['LAT', 'LON']
List of fitted covariate features:
['MN_RADIUS_GF_SECNE', 'MN_RADIUS_GF_SECSE', 'MN_RADIUS_GF_SECSW', 'MN_RADIUS_GF_SECNW']

Validating Cyclone ID(s): 
['AU202021_11U', 'AU202021_15U', 'AU202021_17U', 'AU202021_22U']
Validating Cyclone Name(s):
['Lucas', 'Marian', 'Niran', 'Seroja']

Number of training cyclones used for each run: 93
Validating method: leave one out
Number of validating runs: 4
Prediction at step n ahead: n=  [1, 2, 3, 4, 5, 6, 7, 8]

##################################################
Summarry Table of Scaled Error after 4 Validating Runs.
##################################################


Error Units: scaled between 0 to 1
+---------+---------+---------+---------+---------+---------+---------+---------+---------+
|         |      6h |     12h |    

In [25]:
# Geoplot
if len(path_cov_r34s)==6:
    geoplot(geo_plot_data=path_cov_r34s[-1],plotname='path_cov_r34s.html')
else:
    print('No data for geoplot.')

A geoplot named 'path_cov_r34s.html' was created.


#### 4. Modeling Cyclone Path with Covariates R34s and 'CENTRAL_PRES', 'ENV_PRES'

In [26]:
# Fitting cycpath with all covs
path_cov_all = modelfitting(tscols=cycpath,df_list_of_cyclones=compare, covariates=r34s+pres)

Length = 14
Cyclone number = 1
Cyclone ID: AU202021_11U
Epoch 29: 100%|█████████████████████████████████████████████████| 131/131 [00:18<00:00,  7.08it/s, train_loss=0.000521]
Length = 33
Cyclone number = 2
Cyclone ID: AU202021_15U
Epoch 29: 100%|█████████████████████████████████████████████████| 131/131 [00:17<00:00,  7.31it/s, train_loss=0.000452]
Length = 18
Cyclone number = 3
Cyclone ID: AU202021_17U
Epoch 29: 100%|██████████████████████████████████████████████████| 131/131 [00:17<00:00,  7.29it/s, train_loss=0.00111]
Length = 30
Cyclone number = 4
Cyclone ID: AU202021_22U
Epoch 29: 100%|██████████████████████████████████████████████████| 131/131 [00:18<00:00,  7.20it/s, train_loss=0.00124]


In [27]:
# IMPORTANT: only need to change model name in [] in  'for loop' for reports
# Don't change names of variables on the left hand side.

for fittedmodel in [path_cov_all]:
   ### DON'T CHANGE BELOW CODE
    
    # Assign backtest output of fitted model for reports
    bt_act = fittedmodel[0][0]
    bt_fc = fittedmodel[0][1]
    sc_bt_act = fittedmodel[0][2]
    sc_bt_fc = fittedmodel[0][3]
    bt_error_fit = fittedmodel[0][4]

    # Assign other outputs from fitted model for reports
    target_col = fittedmodel[1]
    traincycount_fit = fittedmodel[2]
    valcycount_fit = fittedmodel[3]
    covariates_fit = fittedmodel[4]
    
    ### DON'T CHANGE ABOVE CODE

In [28]:
# save back test outputs to csv files
bt_csv(modelname='path_cov_all')

In [29]:
# calculate back test set error and save to csv files
bt_set_errors(modelname='path_cov_all')

In [30]:
# summary report
summary_rep()

##################################################
SUMMARY REPORT FOR MODEL EVALUATION
##################################################

Number of fitted target features: 2
List of fitted target features: 
['LAT', 'LON']
List of fitted covariate features:
['MN_RADIUS_GF_SECNE', 'MN_RADIUS_GF_SECSE', 'MN_RADIUS_GF_SECSW', 'MN_RADIUS_GF_SECNW', 'CENTRAL_PRES', 'ENV_PRES']

Validating Cyclone ID(s): 
['AU202021_11U', 'AU202021_15U', 'AU202021_17U', 'AU202021_22U']
Validating Cyclone Name(s):
['Lucas', 'Marian', 'Niran', 'Seroja']

Number of training cyclones used for each run: 93
Validating method: leave one out
Number of validating runs: 4
Prediction at step n ahead: n=  [1, 2, 3, 4, 5, 6, 7, 8]

##################################################
Summarry Table of Scaled Error after 4 Validating Runs.
##################################################


Error Units: scaled between 0 to 1
+---------+---------+---------+---------+---------+---------+---------+---------+---------+
|      

In [31]:
# Geoplot
if len(path_cov_all)==6:
    geoplot(geo_plot_data=path_cov_all[-1],plotname='path_cov_all.html')
else:
    print('No data for geoplot.')

A geoplot named 'path_cov_all.html' was created.


### Model comparison with Paired t-test

In [2]:
# Read in evaluation report files generated from previous sessions

fcycpath = pd.read_csv('bt_set_error_path_nocov.csv')
fcycpath_cov_pres = pd.read_csv('bt_set_error_path_cov_pres.csv')
fcycpath_cov_r34s = pd.read_csv('bt_set_error_path_cov_r34s.csv')
fcycpath_cov_all = pd.read_csv('bt_set_error_path_cov_all.csv')

# Get MAE, RMSE, MSE columns only
no_cov, cov_pres, cov_r34s, cov_all = fcycpath.iloc[:,-3:], fcycpath_cov_pres.iloc[:,-3:], fcycpath_cov_r34s.iloc[:,-3:], fcycpath_cov_all.iloc[:,-3:]

In [3]:
# Generate paired t-test report
from scipy import stats
from tabulate import tabulate

r34s_short = ['r34_ne','r34_se','r34_sw','r34_nw']
errordf = [no_cov, cov_pres, cov_r34s, cov_all]
modelname = ['path_nocov','path_cov_pres', 'path_cov_r34s', 'path_cov_all']
covset = ['none',pres,r34s_short,r34s_short+pres]

# Generate table of mean errors across all models
mean_list=[]
for df in errordf:
    m = df.mean().values
    mean_list.append(m)
mean_df = pd.DataFrame(mean_list, columns= ['MAE','RMSE','MSE'])
mean_df['Model'] = modelname
mean_df['Covariates'] = covset
mean_df = mean_df[['Model','Covariates','MAE','RMSE','MSE']]
print('#'*50)
print('Table of Mean of Error Scores Accross 8 Forecast Points')
print('#'*50)
print(tabulate(mean_df.round(4), headers = 'keys', tablefmt = 'psql'))

# Generate table of Paired t-test
for metrics in ['MAE','RMSE','MSE']:
    p_val = []
    for df in errordf[1:]:
        p = stats.ttest_rel(no_cov[metrics], df[metrics]).pvalue.round(3)
        p_val.append(p)

    pair_ttest = pd.DataFrame([np.repeat(modelname[:1],len(modelname)-1),modelname[1:],p_val],
                             index=['Model_1','Model_2', 'p_value'])
    pair_ttest = pair_ttest.transpose()
    pair_ttest['Significant alpha=0.05'] = np.where(pair_ttest['p_value']<0.05,'yes', 'no')
    print('#'*50)
    print(f"\nPAIRED T-TEST OF {metrics} SCORES AT 95% CONFIDENCE LEVEL")
    print(tabulate(pair_ttest, headers = 'keys', tablefmt = 'psql'))
    print('#'*50+'\n')

##################################################
Table of Mean of Error Scores Accross 8 Forecast Points
##################################################
+----+---------------+----------------------------------------------------------------------+--------+--------+--------+
|    | Model         | Covariates                                                           |    MAE |   RMSE |    MSE |
|----+---------------+----------------------------------------------------------------------+--------+--------+--------|
|  0 | path_nocov    | none                                                                 | 0.1216 | 0.1462 | 0.0309 |
|  1 | path_cov_pres | ['CENTRAL_PRES', 'ENV_PRES']                                         | 0.1448 | 0.1674 | 0.0375 |
|  2 | path_cov_r34s | ['r34_ne', 'r34_se', 'r34_sw', 'r34_nw']                             | 0.1382 | 0.1561 | 0.0334 |
|  3 | path_cov_all  | ['r34_ne', 'r34_se', 'r34_sw', 'r34_nw', 'CENTRAL_PRES', 'ENV_PRES'] | 0.1246 | 0.1423 | 0.02